# Creating a Logistic Regression to Predict Absenteeism

### Import Relavant Libraries

In [135]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

import pickle

### Load the Data

In [5]:
data_preprocessed = pd.read_csv('absenteeism_preprocessed.csv')
data_preprocessed.drop('Unnamed: 0', axis=1, inplace=True)
data_preprocessed.head()

,reason_group_1,reason_group_2,reason_group_3,reason_group_4,day_of_week,month,transportaion_expense_dollars,distance_to_work_miles,age,daily_work_load_average,body_mass_index,education,children,pets,absenteeism_time_hours
0,0,0,0,1,1,7,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,1,7,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,2,7,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,3,7,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,3,7,289,36,33,239.554,30,0,2,1,2


The task is more or less straightforward we will use a logistic regression which will take the reason for absence month of the year day of the week transportation expense distance to work age daily workload average education children and pets of a given employee and will predict their absenteeism we expect that half of those predictors won't have merit to me it seems that the reason for absence will be the most indicative maybe workload will have something to do with it as well since the busier person is the less he or she will want to skip work finally children and pets together with distance from work should also have something to do with absenteeism if your child or pet is sick at home you'll have to go home take them to the doctor and get them back which will be much more time consuming than a simple said the doctor OK we have a good idea what to expect the nice thing about regressions is that the model itself will give us a fair indication of which variables are important for the analysis 

### Create the Targets

More or less straightforward we will use a logistic regression which will take the reason for absence month of the year day of the week transportation expense distance to work age daily workload average education children and pets of a given employee and will predict their absenteeism we expect that half of those predictors won't have merit to me it seems that the reason for absence will be the most indicative maybe workload will have something to do with it as well since the busier person is the less he or she will want to skip work finally children and pets together with distance from work should also have something to do with absenteeism if your child or pet is sick at home you'll have to go home take them to the doctor and get them back which will be much more time consuming than a simple said the doctor OK we have a good idea what to expect the nice thing about regressions is that the model itself will give us a fair indication of which variables are important for the analysis 

In [8]:
data_preprocessed['absenteeism_time_hours'].describe()

count    700.000000
mean       6.761429
std       12.670082
min        0.000000
25%        2.000000
50%        3.000000
75%        8.000000
max      120.000000
Name: absenteeism_time_hours, dtype: float64

Cutoff = median = 3 hours. Anyone more than 3 hours is excessively absent (assign value 1) and less is moderately so (assign value 0). These are targets in supervised learning, the values we're aiming for. Our task is to predict whether we'll get a 0 or a 1.

In [10]:
cutoff = data_preprocessed['absenteeism_time_hours'].median()
targets = np.where(data_preprocessed['absenteeism_time_hours'] > cutoff, 1, 0)
targets

array([1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0,

result is an np array with ones and zeros. Add this as a new column 'excessive_absenteeism'.

In [12]:
data_preprocessed['excessive_absenteeism'] = targets
data_preprocessed.head()

,reason_group_1,reason_group_2,reason_group_3,reason_group_4,day_of_week,month,transportaion_expense_dollars,distance_to_work_miles,age,daily_work_load_average,body_mass_index,education,children,pets,absenteeism_time_hours,excessive_absenteeism
0,0,0,0,1,1,7,289,36,33,239.554,30,0,2,1,4,1
1,0,0,0,0,1,7,118,13,50,239.554,31,0,1,0,0,0
2,0,0,0,1,2,7,179,51,38,239.554,31,0,0,0,2,0
3,1,0,0,0,3,7,279,5,39,239.554,24,0,2,0,4,1
4,0,0,0,1,3,7,289,36,33,239.554,30,0,2,1,2,0


We've mapped the data into two classes now.


Using the median as a cutoff line is numerically stable and rigid that's because by using the media we have implicitly balanced the data set roughly half of the targets are zeros while the other half ones as you may remember this will prevent our model from learning to output one of the two classes exclusively thinking it did very well in order to prove that let's divide the number of targets that are ones by the total number of targets the number of targets that are ones can be found by summing up all values of targets while the total number of targets is simply the shape on axis 0 the result is around 0.46 so around 46% of the targets are ones thus around 54% of the targets are zeros Please remember that when balancing your data set the two classes needed represent 50% of the sample exactly usually 6040 split will work equally well for logistic regression but that's not true for other algorithms such as neural networks for my personal experience a balance of 45 to 55% is almost always sufficient so our results will do for this exercise let's proceed noting that our two groups have been distributed roughly equally finally let's drop the absenteeism 19 hours from the data frame since we won't be needing it I'll call the new variable data with targets 

In [14]:
targets.sum() / targets.shape[0]

0.45571428571428574

In [15]:
data_with_targets = data_preprocessed.drop(['absenteeism_time_hours'], axis=1)
data_with_targets.head()

,reason_group_1,reason_group_2,reason_group_3,reason_group_4,day_of_week,month,transportaion_expense_dollars,distance_to_work_miles,age,daily_work_load_average,body_mass_index,education,children,pets,excessive_absenteeism
0,0,0,0,1,1,7,289,36,33,239.554,30,0,2,1,1
1,0,0,0,0,1,7,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,2,7,179,51,38,239.554,31,0,0,0,0
3,1,0,0,0,3,7,279,5,39,239.554,24,0,2,0,1
4,0,0,0,1,3,7,289,36,33,239.554,30,0,2,1,0


### Selecting the Inputs for the Regression

### Backward elimination performed 
Along with dropping the original absenteeism_time_hours

In [18]:
columns_to_eliminate = ['day_of_week', 'daily_work_load_average', 'distance_to_work_miles']
data_with_targets = data_with_targets.drop(columns_to_eliminate, axis=1)
data_with_targets.head()

,reason_group_1,reason_group_2,reason_group_3,reason_group_4,month,transportaion_expense_dollars,age,body_mass_index,education,children,pets,excessive_absenteeism
0,0,0,0,1,7,289,33,30,0,2,1,1
1,0,0,0,0,7,118,50,31,0,1,0,0
2,0,0,0,1,7,179,38,31,0,0,0,0
3,1,0,0,0,7,279,39,24,0,2,0,1
4,0,0,0,1,7,289,33,30,0,2,1,0


In [19]:
data_with_targets.iloc[:, :-1]

,reason_group_1,reason_group_2,reason_group_3,reason_group_4,month,transportaion_expense_dollars,age,body_mass_index,education,children,pets
0,0,0,0,1,7,289,33,30,0,2,1
1,0,0,0,0,7,118,50,31,0,1,0
2,0,0,0,1,7,179,38,31,0,0,0
3,1,0,0,0,7,279,39,24,0,2,0
4,0,0,0,1,7,289,33,30,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,5,179,40,22,1,2,0
696,1,0,0,0,5,225,28,24,0,1,2
697,1,0,0,0,5,330,28,25,1,0,0
698,0,0,0,1,5,235,32,25,1,0,0


it is time to select the inputs for our regression we will use the pandas method i loc
select all rows as inputs  all columns except for excessive absenteeism right if i have to select all rows i'll simply leave collins as the first argument regarding the columns i want to keep the 1st 14 since excessive absenteeism is the column with index

In [21]:
unscaled_inputs = data_with_targets.iloc[:, :-1]

### Standarize the Data

back it is time to standardize our data we have discussed why standardization is important There are several ways to perform standardization what's important relevant module from SK learn 

Here is a big big problem which you are not going to like when we standardize the inputs we also standardize the dummies this is bad practice because when we standardize we lose the whole interpretability of a dummy if we had left them as zeros and ones we could have said for a unit change it is 7.2 times more likely that a person will be excessively absent a unit change in the dummy variable universe means a change from disregarding this dummy to taking only this dummy into account so if the the reason is reason one we would have said finally we can declare from disregarding this dummy to taking only this dummy into account so if the reason is reason one we would have said it is around 7.7 times more likely than a person will be absent compared to no reason given whenever we standardize the reasons and now a unit change is completely uninterpretable the predictive power of the model is still valid and it is a good classifier but we don't know how the different reasons compare this is a problem because those are the most important features this brings us to a correction of our code that thing we had all those checkpoints maybe they'll help us do that effortlessly I'll go back to the part where we standardize the data and put all the code in comments since we won't be needing this now I'll copy paste some code that I've prepared prior to this lecture the idea is that this is a custom scaler based on the standard scaler from sklearn however when we declare the scalar project there's an extra argument columns to scale so our custom scaler will not standardize all of inputs but only the ones we choose in this way we will be able to preserve the dummies untouched in practice we would avoid this step by standardizing prior to creating the dummies but we didn't do it this time so we can show you yet another nifty tool so let's see how this will work 1st we have the custom scalar class is no different than the standard scaler in the way it works you don't necessarily need to understand the code all you need to know is how to use it 2nd we must choose the columns to be scaled to see what columns we've got we must return to the last checkpoint before we standardize this seems to be the unscaled input variable so let's see what its column values are alright we will create a new variable called columns to scale that will contain the names of the features we'd like to scale therefore we will omit the dummy variables from this list

In [24]:
# absenteeism_scaler = StandardScaler()

the object we just created will be used to scale our data in other words it will subtract the mean and divide by the standard deviation from each point variable wise again if you feel uncomfortable with this idea be sure to check the standardization lessons the next step is to fit our input data. The following line calculates and store the mean and std deviation. Ans absenteeism scaler will contain the mean and std info so we get a new data we can standarize it in the same way.

In [26]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

class CustomScaler(BaseEstimator,TransformerMixin): 

    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        self.scaler = StandardScaler(copy=copy, with_mean=with_mean, with_std=with_std)
        self.columns = columns
        self.mean_ = None
        self.var_ = None
        self.copy = copy
        self.with_mean = with_mean
        self.with_std = with_std
    
    # the fit method, which, again based on StandardScale
    
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self

    def transform(self, X, y=None, copy=None):
        init_col_order = X.columns
        
        # scale all features that you chose when creating the instance of the class
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        
        # declare a variable containing all information that was not scaled
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [27]:
unscaled_inputs.columns.values

array(['reason_group_1', 'reason_group_2', 'reason_group_3',
       'reason_group_4', 'month', 'transportaion_expense_dollars', 'age',
       'body_mass_index', 'education', 'children', 'pets'], dtype=object)

In [28]:
# columns_to_scale = ['month', 'transportaion_expense_dollars', 'age', 'body_mass_index', 'education', 'children', 'pets']
columns_to_omit = ['reason_group_1', 'reason_group_2', 'reason_group_3', 'reason_group_4', 'education']
columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]

absenteeism_scaler = CustomScaler(columns_to_scale)

In [29]:
absenteeism_scaler.fit(unscaled_inputs)

C:\Users\shree\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3785: FutureWarning: The behavior of DataFrame.var with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return var(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


CustomScaler(columns=['month', 'transportaion_expense_dollars', 'age',
                      'body_mass_index', 'children', 'pets'])

We've created a scaling mechanism. In order to apply it we must use transform.

In [31]:
scaled_inputs = absenteeism_scaler.transform(unscaled_inputs)
scaled_inputs

,reason_group_1,reason_group_2,reason_group_3,reason_group_4,month,transportaion_expense_dollars,age,body_mass_index,education,children,pets
0,0,0,0,1,0.182726,1.005844,-0.536062,0.767431,0,0.880469,0.268487
1,0,0,0,0,0.182726,-1.574681,2.130803,1.002633,0,-0.019280,-0.589690
2,0,0,0,1,0.182726,-0.654143,0.248310,1.002633,0,-0.919030,-0.589690
3,1,0,0,0,0.182726,0.854936,0.405184,-0.643782,0,0.880469,-0.589690
4,0,0,0,1,0.182726,1.005844,-0.536062,0.767431,0,0.880469,0.268487
...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,-0.388293,-0.654143,0.562059,-1.114186,1,0.880469,-0.589690
696,1,0,0,0,-0.388293,0.040034,-1.320435,-0.643782,0,-0.019280,1.126663
697,1,0,0,0,-0.388293,1.624567,-1.320435,-0.408580,1,-0.919030,-0.589690
698,0,0,0,1,-0.388293,0.190942,-0.692937,-0.408580,1,-0.919030,-0.589690


Summies are untouched

In [33]:
scaled_inputs.shape

(700, 11)

All input data has been standardized.

## Split the Data Into Train & Test and Shuffle
Test accuracy of data it has never seen before. Shuffle data to remove all kinds of dependency from the order of the data.

### Split

In [37]:
train_test_split(scaled_inputs, targets)

[     reason_group_1  reason_group_2  reason_group_3  reason_group_4     month  \
 208               0               0               1               0 -0.388293   
 185               0               0               0               1 -0.673803   
 246               0               0               0               1  0.182726   
 314               1               0               0               0  1.039256   
 609               0               0               0               1 -1.244823   
 ..              ...             ...             ...             ...       ...   
 18                1               0               0               0  0.182726   
 444               0               0               0               1 -0.102784   
 365               0               0               0               1 -1.530333   
 125               0               0               0               1 -1.530333   
 377               1               0               0               0 -1.244823   
 
      transpor

We receive 4 arrays: a training data with inpust, a testing data with input, a training data with targets, a testing data with targets.

In [39]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets)

In [40]:
print(x_train.shape, y_train.shape)

(525, 11) (525,)


In [41]:
print(x_test.shape, y_test.shape)

(175, 11) (175,)


Training Input is 525 observation along 14 variables and 525 targtes (excessive_absenteeism). 75%. 25% is testing.

In [43]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, train_size = 0.8, random_state = 20) # 80-20 split; sudo-random always shuffle but in the same way shuffle = True by def

In [44]:
print(x_train.shape, y_train.shape)

(560, 11) (560,)


In [45]:
print(x_test.shape, y_test.shape)

(140, 11) (140,)


## Logistic Regression wit SKlearn

### Training the Model

In [48]:
reg = LogisticRegression()
reg.fit(x_train, y_train) # Output is all parameter

LogisticRegression()

In [49]:
reg.score(x_train, y_train) # R 78% accuracy; model learned to classify 78% of the observations correctly 

0.7732142857142857

### Manually Check Accuracy

That's the target so if we want to find the accuracy of a model manually we should find the outputs and compare them with the targets let's do that in order to find the model outputs we will use a simple ask learned method it is called predict so if I write Reg dot predict X-ray this method will find the predicted outputs of the regression the model itself is contained in the variable Reg and we are choosing to predict the output to associated with the training inputs and contained an X training let's store all that in a new variable called model outputs and see what's inside

In [51]:
model_outputs = reg.predict(x_train) #  (Training Data) - Predictions of model
model_outputs

array([0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,

In [52]:
model_outputs == y_train

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True, False, False,  True,  True,  True,  True,
       False,  True, False,  True, False, False,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False, False, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
       False,  True, False,  True,  True, False, False, False,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
       False,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,

In [53]:
np.sum(model_outputs == y_train) # Predicted correctly

433

In [54]:
model_outputs.shape[0] # Total observations

560

In [55]:
np.sum(model_outputs == y_train) / model_outputs.shape[0] # Exact same - behind the scene

0.7732142857142857

### Finding the Intercept and Coefficients
in this Python SQL tableau integration the ultimate goal would be to create a function which could easily and reliably predict values from within tableau since tableau is a nice looking manager friendly software that's the place where the end users of our analysis will likely take advantage of our model regression analysis no matter if linear or nonlinear is about determining certain coefficients or wait which we apply to the inputs to obtain a final result so to use this logistic regression model outside of Python we must get our hands on the coefficients and the intercept moreover in order to interpret this logistics model we still need to do so let's get on with that we intercepted we found very easily using the intercept method

In [57]:
reg.intercept_

array([-1.6469898])

In [58]:
reg.coef_

array([[ 2.80000644,  0.95174778,  3.1140605 ,  0.83835931,  0.15897713,
         0.60513709, -0.16990589,  0.27998236, -0.21017416,  0.34842434,
        -0.27721907]])

In [59]:
feature_name = unscaled_inputs.columns.values # variables corresponding to the coefficients
feature_name

array(['reason_group_1', 'reason_group_2', 'reason_group_3',
       'reason_group_4', 'month', 'transportaion_expense_dollars', 'age',
       'body_mass_index', 'education', 'children', 'pets'], dtype=object)

In [60]:
summary_table = pd.DataFrame(columns = ['Feature Name'], data = feature_name)
summary_table['Coefficient'] = np.transpose(reg.coef_)
summary_table

,Feature Name,Coefficient
0,reason_group_1,2.800006
1,reason_group_2,0.951748
2,reason_group_3,3.114061
3,reason_group_4,0.838359
4,month,0.158977
5,transportaion_expense_dollars,0.605137
6,age,-0.169906
7,body_mass_index,0.279982
8,education,-0.210174
9,children,0.348424


In [61]:
# Add intercept
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table

,Feature Name,Coefficient
0,Intercept,-1.646990
1,reason_group_1,2.800006
2,reason_group_2,0.951748
3,reason_group_3,3.114061
4,reason_group_4,0.838359
5,month,0.158977
6,transportaion_expense_dollars,0.605137
7,age,-0.169906
8,body_mass_index,0.279982
9,education,-0.210174


Remind you that the coefficients are also called weights while the intercept bias these notions are useful because the weights show how we weigh a certain input the closer they are to 0 the smaller the weight and alternatively the further away from zero no matter if positive the bigger the weight of this feature note that this is something which is true for our model but it's not universally true it holds only for models where all variables are on the same scale such as the one we just built there are coefficient values and standardized coefficient values these standardized coefficients are basically the coefficient values of a regression where all variables have been standardized other packages and software include the standardized coefficients because they allow for a simple and easy to understand comparison between the variables since in such cases the features are standardized they all have a variance of one or the same scale and whenever the scale is standard or the same that is we can simply say whatever weight is bigger its corresponding feature is more important for machine learning purposes and prediction in general we usually standardize the variables like we did now OK now another notion we must emphasize is that whenever we are dealing with the logistic regression the coefficients we are predicting are the so-called log odds this is a consequence of the choice of model logistics you are nothing but a linear function odds are later transformed into zeros and ones let's make this clear here's the logistic regression equation therefore all the coefficients that we have referred to the log odds so to make them more interpretable let's find the exponentials of these coefficients I'll create a new series that our data frame called odds ratio odds ratio is the correct term for what we will get after we find the exponentials of the locations it is equal to NP Exp of summary table dot coefficient

## Interpreting the Coefficients

In [64]:
summary_table['Odds_ratio'] = np.exp(summary_table.Coefficient)

In [65]:
summary_table.sort_values('Odds_ratio', ascending = False)

,Feature Name,Coefficient,Odds_ratio
3,reason_group_3,3.114061,22.512270
1,reason_group_1,2.800006,16.444753
2,reason_group_2,0.951748,2.590233
4,reason_group_4,0.838359,2.312570
6,transportaion_expense_dollars,0.605137,1.831503
10,children,0.348424,1.416833
8,body_mass_index,0.279982,1.323106
5,month,0.158977,1.172311
7,age,-0.169906,0.843744
9,education,-0.210174,0.810443


And so how can we interpret them if the coefficient is around 0 or its odds ratio is close to one this means that the corresponding feature is not particularly important The reasoning in terms of weights is that a weight of 0 implies that no matter the feature value we will multiply it by zero and the whole result will be 0 the meaning in terms of odds ratios is the following for one unit change in the standardized feature the odds increase by a multiple equal to the odds ratio So if the odds ratio is 1 then the odds don't change at all for example if the odds are 5 to 1 and the odds ratio is 2 we would say that for one unit change the odds change from 5:00 to 1:00 to 12:50 because we multiplied them by the odds ratio alternatively if the odds ratio is 0.2 the odds would change to 1:00 to 1:00 when the ratio is 1 we don't have a change as multiplication with the number one keeps things equal this makes sense as the odds ratio was one whenever the way is 0
The average daily workload is -0.03 so almost zero and it's odd ratio is 0.97 so almost 1 so this feature is almost useless for our model and with or without it the result would likely be the same do we have other variables that could fall into this category of course day of the week and distance to work to be honest this is a bit surprising to me the justifying my mistake in logic this is the time to know that they may not necessarily be useless a more accurate statement is that given all features these seem to be the ones that make no difference we will keep the features for now but consider dropping them later on all right what else could we see we've got the four reasons for absence which are the most important predictors when we were creating the dummies the one we dropped was reason zero reason 0 represented a situation when a person was absent but no particular reason was given therefore the base model the case where there is no reason AKA reason 0 from the coefficients it seems that whenever a person has stated any reason we have a much higher chance of getting excessive absence


Since dummies weren't standardize, we can interpret it.

Features I'll just remind you that the further away from zero coefficient is the bigger it's importance so by looking at the coefficients table we will notice that the most strongly pronounced features seem to be the four reasons for absence transportation expenses and whether a person has children pets and education note that pet and education are at the bottom of the table but they're weights are still far away from zero but they are indeed important we can carry on in this way finishing with the daily work load distance to work and day of the week which seemed to have the smallest impact their weight is almost 0 so regardless of the particular values they will barely affect our model what about the reasons we said that the base model includes no reason but what is the impact of the various reasons I'll quickly recap with the five reason variable stand for reason 0 or no reason which is the baseline model reason one which comprises of various diseases reason 2 relating to pregnancy and giving birth the reason 3 regarding poisoning and peculiar reasons not categorized square and reason for which relates to light diseases in the light of this clarification we can easily understand our coefficients the most crucial reason for excessive absence is poisoning not much of a surprise there and if you are poison you just won't go to work the weight means the odds of someone that you excessively absent after being poisoned are 20 times higher than when no reason was reported OK clear another very important reason seems to be #1 or various diseases I've called this the normal absenteeism case you got sick you skipped work no drama a person who has reported this is 14 times more likely to be excessively absent than a person who didn't specify a reason then we have pregnancy and giving birth I particularly like this one because it's a prominent cause of absenteeism but at the same time it is way less pronounced than reasons one and three my explanation for this is a woman is pregnant she goes to the gynecologist gets her regular pregnancy check and comes back to work nothing excessive about that from time to time there are some emergencies but from the odds ratio we can verify that it's only around 2 times more likely to be excessively absent in the base model all right after that we've got transportation expense this is the most important non dummy feature in the model side here's the problem it is one of our standardized variables we don't have direct interpretability of it it's odds ratio implies that for one standardized unit or for one standard deviation increase in transportation expense it is close to twice as likely to be excessively absent this is the main drawback of standardization standardized models almost always yield higher accuracy because the optimization algorithms work better in this way machine learning engineers prefer models with higher accuracy so they normally go for sanitization econometricians and statisticians however prefer less accurate but more interpretable models because they care about the underlying reasons behind different phenomena data scientists may be in either position sometimes they need higher accuracy other times they must find the main drivers of a problem so it makes sense to create two different models one with standardized features and one without them and then draw insights from both however should we opt for predicting values we definitely prefer higher accuracy so standardization is more often 
Negative proficient I'll go for pet pet is a continuous variable it's odds ratio is 0.7 so for each additional standardized unit of pet the odds are 1 minus its odds ratio or 24% lower than the base bottle model that is no pet 1 - 0.759676 equals 24% one explanation may be if you have several pets you're probably not taking care of them on your own not being solely responsible for them implies somebody else can take them to the doctor if something's wrong finally I want to make a note on the intercept it is used to get more accurate predictions but there's no specific meaning attached to it that's why in machine learning you can say that it calibrates the model and you could also call it a bias without it each prediction would be off by precisely that value


## Backward Elimination and Simplifying Model

columns that have no impact or predictive power. Month value is useful even if it doesn't add predictive power. 

Removing all features which have close to no contribution to the model usually when we have the P values of variables we get rid of all coefficients with P values above 0.05 when learning with SK learned we don't have P values because we don't necessarily need them the reasoning of the engineers created the package is that if the weight is small enough it won't make a difference anyway and we trust their work OK so I say that if we remove these variables the rest of our model should not really change in terms of coefficient values let's go back to the checkpoint when we created the targets this was our last data frame manipulation step before we started standardizing i'll take this checkpoint data with targets and drop the three features we were just discussing guessing

Slight diff in accuray ~ 0.02% but coefficient is the same. so, it worked

## Testing the Model

Our algorithm has seen this training data many times in fact thousands of times during the training process so it has learned to model it's quite well however it may fail miserably when provided with new data as we said earlier we should test it on data it has never seen it is time to use the test data this will also be the end of the machine learning part that's because testing is done only once and at the very end of the machine learning process why is that well some researchers are looking at the testing accuracy and then tweaking the model a bit to get better test accuracy however if you do this operation enough times what will this an iterative process in which you change some parameters based on a function the accuracy in this case but that's basically the definition of the machine learning training process so instead of testing you will be using the test data to train a bit more but this time manually this makes the test data set useless because you are not really testing the take away is that once we test we are not conceptually allowed to touch the model anymore all right let's finally test it the first task on our hands is to find the accuracy in the same manner as before

In [112]:
reg.score(x_test, y_test)

0.75

Based on data the model has never seen before, 75% of the time the model will correctly predict. Always less than train data otherwise made a mistake.

20% lower than the train accuracy something like 10% or even 20% lower this would mean that our model overfitted it learned the train data very well but is prone to fail in real life without a small percentage difference between the train and test accuracy they're in neither case so let's focus on the present.

In [121]:
predicted_proba = reg.predict_proba(x_test)
predicted_proba

array([[0.71342516, 0.28657484],
       [0.5873216 , 0.4126784 ],
       [0.44016153, 0.55983847],
       [0.78163061, 0.21836939],
       [0.08407928, 0.91592072],
       [0.3348226 , 0.6651774 ],
       [0.29971206, 0.70028794],
       [0.13112385, 0.86887615],
       [0.78627908, 0.21372092],
       [0.74906578, 0.25093422],
       [0.49395555, 0.50604445],
       [0.22492002, 0.77507998],
       [0.07135527, 0.92864473],
       [0.73173354, 0.26826646],
       [0.30957854, 0.69042146],
       [0.54726422, 0.45273578],
       [0.55051921, 0.44948079],
       [0.53926379, 0.46073621],
       [0.40197149, 0.59802851],
       [0.05365482, 0.94634518],
       [0.70030387, 0.29969613],
       [0.78163061, 0.21836939],
       [0.42028246, 0.57971754],
       [0.42028246, 0.57971754],
       [0.24801464, 0.75198536],
       [0.74567806, 0.25432194],
       [0.51026557, 0.48973443],
       [0.8569309 , 0.1430691 ],
       [0.20365204, 0.79634796],
       [0.78163061, 0.21836939],
       [0.

In [127]:
predicted_proba[:, 1] # Probability of being 1

array([0.28657484, 0.4126784 , 0.55983847, 0.21836939, 0.91592072,
       0.6651774 , 0.70028794, 0.86887615, 0.21372092, 0.25093422,
       0.50604445, 0.77507998, 0.92864473, 0.26826646, 0.69042146,
       0.45273578, 0.44948079, 0.46073621, 0.59802851, 0.94634518,
       0.29969613, 0.21836939, 0.57971754, 0.57971754, 0.75198536,
       0.25432194, 0.48973443, 0.1430691 , 0.79634796, 0.21836939,
       0.36947677, 0.67913195, 0.68508325, 0.52870791, 0.21836939,
       0.53505228, 0.22144744, 0.73673169, 0.40500758, 0.60504297,
       0.21072119, 0.45227108, 0.23749326, 0.39847178, 0.82763577,
       0.56771922, 0.69120847, 0.28657484, 0.2192347 , 0.2032712 ,
       0.57634482, 0.32954238, 0.6651774 , 0.26937528, 0.83323682,
       0.43484145, 0.88365871, 0.23125087, 0.33433749, 0.34451397,
       0.69915101, 0.6549938 , 0.29244583, 0.79186052, 0.20752232,
       0.26838009, 0.08710411, 0.22144744, 0.73215219, 0.30536526,
       0.22144744, 0.2900789 , 0.90443841, 0.46065771, 0.60175

In [129]:
predicted_proba.shape

(140, 2)

Zero and one we can get the probability of an output being zero or one there is an SK learn method called predict proba so let's predicted proba be equal to Reg dot predict proba of X test what we get is a 140 by two array there are 140 test observations and two columns the first column shows the probability our model assigned to the observation being zero and the second the probability of the model assigned to the observation being one that's why summing any 2 numbers were regionally will give you an output of one see OK what we are interested in is a probability of excessive absenteeism right so the probability of getting one therefore we can simply slice out all values from the second column this will give us the probabilities of excessive absenteeism and this result is much cooler than simply zeros or ones in reality logistic regression models calculate these probabilities in the background if the probability is below 0.5 it places a 0 otherwise A1 there are other considerations we can make and other ways to see how good our model actually is however we won't be diving too deep instead we want to get back to the integration from this point on we will take several important steps first we will save our model so we can use it later on we don't need to train it every time we just need to determine the weights once and then save them for later use second we will create our own module so that our less technical colleagues could take advantage of this model too finally we will get completely new data classify it pass it through SQL and then analyze it in tableau 

## Save the Model

there is little use in running a machine learning algorithm without applying it to new data usually when you are not simply practicing your ML skills you would train to model once and then save it saving a model is the process of creating a file that will contain all the information regarding the machine learning roughly speaking we want to create a file that will store the following information this machine learning model is a logistic regression it has these and these coefficients that intersect the random state that was chosen for the shuffling was 20 and so on now if you pay close attention to this exercise you would quickly realize that the object rec which was an instance of the SK learn logistic regression class contains all this information in fact this is the object we use to find the intercept coefficients and accuracy therefore it should not come as a surprise that leaving the model is equivalent to saving The Reg object this is precisely what we will do 

We'll use the method pickling to convert Python object (reg) into a character stream (will contain sufficient info) and later can be converted to python object by unpickling. File is less than 1KB! or send it by email and any one can unpickle it and intergrate it in their code.

In [139]:
with open('model', 'wb') as file:
    pickle.dump(reg, file)

Finally there are several important points that need to be made first the file name I will set that to model as it basically contains the model second we've got WB that stands for right bites conversely when we unpickle we will use RB or read bytes 3rd we've got the dump method when we pickle we dump the information in a file when we unpickle we load it in the dump method we specify the object to be dumped and the rest is just python syntax alright finally there is one more subtlety we must save the absenteeism scaler too the absenteeism scalar object was used to standardize all numerical when i share variables what it did was store the columns to scale as well as the mean and the standard deviation of each feature to further explain why we need to pickle the scaler think that up until now our code was heavily dependent on training data without training data the machine learning could not be executed at all but once the countless trained and we have obtained the coefficients we can save it as we just have in this way we are basically separating the model from the training data for good analogically the information in the absenteeism scaler is needed to preprocess any new data using the same rules as the ones applied to training data thus we must pickle the scaler too use the same pickling code to pickle the absenteeism scalar object in a file called scaler

In [141]:
with open('scalar', 'wb') as file:
    pickle.dump(absenteeism_scaler, file)

Other ways of saving are Joblib (part of SciPy) and JSON, each with its own pros and cons. Pickle is the standard tool for serialization and deserialization (convert a Python object into strings of characters) and back.

Pickling is strictly associated with a single python version. Not secure to unpickle from unauthorized and untrusted source.